In [1]:
import os, glob
import random
import nltk
import unicodedata
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn_extra.cluster import KMedoids
from sklearn.neighbors import KernelDensity
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.metrics import silhouette_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize
from scipy.fftpack import dct
import umap
import matplotlib.pyplot as plt
import re, textwrap
import logging
# logging.basicConfig(filename='./kmeankde.log',
#                             filemode='a',
#                             format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
#                             datefmt='%H:%M:%S',
#                             level=logging.DEBUG)
# logging.debug("test")
# logging.warning("test")
from IPython.display import clear_output

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/mao919/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/mao919/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
#SkipThoughts stuff
import torch
from torch.autograd import Variable
import sys
sys.path.append('skip-thoughts.torch/pytorch')
from skipthoughts import UniSkip
dir_st = 'data/skip-thoughts'

from nltk.corpus import stopwords

In [3]:
#Universal Embeddings Stuff
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import tensorflow_hub as hub
from tensorflow.python.framework.ops import disable_eager_execution

disable_eager_execution()

module_url = "https://tfhub.dev/google/universal-sentence-encoder/2"
embed = hub.Module(module_url)

tf.logging.set_verbosity(tf.logging.ERROR)

Instructions for updating:
non-resource variables are not supported in the long term


In [4]:
#Sentence Transformer stuff
from sentence_transformers import SentenceTransformer
transformer = SentenceTransformer('bert-base-nli-mean-tokens')

In [2]:
contractions = { 
"ain't": "am not / are not / is not / has not / have not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is / how does",
"I'd": "I had / I would",
"I'd've": "I would have",
"I'll": "I shall / I will",
"I'll've": "I shall have / I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

In [3]:
contractions_re = re.compile('(%s)' % '|'.join(contractions.keys()))
def expand_contractions(s, contractions_dict=contractions):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, s)

def preprocessed_list_of_sentences(doc):
    re.sub(r'(?<=[.,!?;:])(?=[^\s])', r' ', doc)
    sentences = sent_tokenize(doc)    
    sentences = [expand_contractions(i) for i in sentences]
    sentences = [re.sub('\n', ' ', i) for i in sentences]
    sentences = list(map(lambda s: unicodedata.normalize('NFKD', s).strip().lower(), sentences))
    sentences = list(map(lambda s: re.sub(r"\s+", " ", s), sentences))
    filtered_sentences = filter(lambda x: len(word_tokenize(x)) > 3, sentences)
    return list(filtered_sentences)

def extract_governance(file):
    with open(file, 'r', encoding='utf-8', errors='ignore') as f:
        section = f.read()
  
    search = re.search("<governance>[\s\S]*?<\/governance>", section)
    if search is None:
        search = re.search("<Governance>[\s\S]*?<\/governance>", section)
  
    section = search.group(0)
    section = [line for line in section.split('\n') if len(line) > 20]
    section = '\n'.join(section)
  
    return section

def extract_overview(file):
    with open(file, 'r', encoding='utf-8', errors='ignore') as f:
        section = f.read()
  
    search = re.search("<overview>[\s\S]*?<\/overview>", section)
    if search is None:
        search = re.search("<Overview>[\s\S]*?<\/overview>", section)
  
    section = search.group(0)
    section = [line for line in section.split('\n') if len(line) > 20]
    section = '\n'.join(section)
  
    return section

def extract_strategy(file):
    with open(file, 'r', encoding='utf-8', errors='ignore') as f:
        section = f.read()

  # remove all the segments inside </overview> tags
    start = section.find("</overview>")
    section = section[start+11:]
    while start != -1:
        start = section.find("<overview>")
        end   = section.find("</overview>")
        substring = section[start:end+11]
        section = section.replace(substring, '')
    

    start, end = 0, 0
  # remove all segments inside <governance> tags
    while start != -1:
        start = section.find("<governance>")
        end   = section.find("</governance>")
        substring = section[start:end+13]
        section = section.replace(substring, '')

    # get rid of short lines  
    section = [line for line in section.split('\n') if len(line) > 20]
    section = '\n'.join(section)

    return section

In [4]:
from sklearn.neighbors import NearestCentroid
def summaryFromClusters(model, points, sentences, method='kmeans'):
    '''
    poinst: shape (n_samples, n_features)
    return closest: 1dim array of indices of closest to each cluster center
    '''
    if method == 'kmedoids':
        indices = model.medoid_indices_
        summary = [sentences[i] for i in indices]
        return ' '.join(summary)
    
    elif method == 'kmeans':
        seen = set()
        seen_add = seen.add
        closest_uniques = []
        try:
            closest, _ = pairwise_distances_argmin_min(model.cluster_centers_, points)
        except AttributeError:
            clf = NearestCentroid()
            clf.fit(points, model.labels_)
            centroids = clf.centroids_
            closest, _ = pairwise_distances_argmin_min(centroids, points)

        closest_uniques = [x for x in closest if not (x in seen or seen_add(x))]
        closest_uniques.sort()
        summary = [sentences[i] for i in closest_uniques]
        return ' '.join(summary)

def silhouette_scoring(estimator, X):
    labels = estimator.fit_predict(X)
    score = silhouette_score(X, labels)
    return score

In [8]:
def prepare_skipThoughtVocab(doc):
    sentences = preprocessed_list_of_sentences(doc)
    #Preparing Vocab
    wordtokenized_sentences = [word_tokenize(sent) for sent in sentences]
    vocab = set(sum(wordtokenized_sentences, []))

    token2id = {}
    for i, token in enumerate(vocab):
        if token not in token2id.keys():
            token2id[token] = i + 1
    for j in range(len(sentences)):
        wordtokenized_sentences[j] += ['<eos>']
  
    token2id['<eos>'] = len(vocab) + 1

    sentence_ids = [[token2id[tok] for tok in sent if tok.isalpha()] for sent in wordtokenized_sentences]
    MAX_LEN = len(max(sentence_ids, key=len)) if len(sentence_ids) > 0 else 0
    for i in range(len(sentence_ids)):
        sentence_ids[i] += [0]* (MAX_LEN - len(sentence_ids[i])) 

    return sentences, sentence_ids, vocab

def SkipThoughtsSum(section, n_sents=50, method='kmeans'):
    sentences, sentence_ids, vocab = prepare_skipThoughtVocab(section)
    uniskip = UniSkip(dir_st, vocab)
    input = torch.LongTensor(sentence_ids)
    output = torch.zeros(input.size(0), 2400, dtype=torch.long)
    MAX_LEN = len(max(sentence_ids, key=len)) if len(sentence_ids) > 0 else 0
    for i in range(input.size(0)):
        output[i:i+1] = uniskip(input[i:i+1], lengths=[MAX_LEN])
  
    if output.size(0) == 0:
        logging.warning("strategy is empty")
        return ''
    if n_sents > len(sentences):
        logging.warning("Overview larger than strategy")
        return 'Overview is larger than strategy'
    
    if method == 'kmedoids':
        kmedoids = KMedoids(n_clusters=n_sents)
        kmedoids.fit(output.numpy())
        summary = summaryFromClusters(kmedoids, output.numpy(), sentences, method='kmedoids')
    elif method == 'kmeans':
        output = normalize(output.numpy(), norm='l2')
        clusterer = KMeans(n_clusters=n_sents)
        clusterer.fit(output)
        summary = summaryFromClusters(clusterer, output, sentences, method='kmeans')
    
    return summary

    
def Aggl_SkipThoughtsSum(section, n_sents):
    sentences, sentence_ids, vocab = prepare_skipThoughtVocab(section)
    uniskip = UniSkip(dir_st, vocab)
    input = torch.LongTensor(sentence_ids)
    output = torch.zeros(input.size(0), 2400, dtype=torch.long)
    MAX_LEN = len(max(sentence_ids, key=len)) if len(sentence_ids) > 0 else 0
    for i in range(input.size(0)):
        output[i:i+1] = uniskip(input[i:i+1], lengths=[MAX_LEN])
  
    if output.size(0) == 0:
        return ''
    if n_sents > len(sentences):
        logging.warning("Overview larger than strategy")
        return 'Overview is larger than strategy'
    
    output = normalize(output.numpy(), norm='l2')
    clusterer = AgglomerativeClustering(n_clusters=n_sents, affinity='euclidean', linkage='ward')
    clusterer.fit(output)
    
    summ = summaryFromClusters(clusterer, output, sentences)
    return summ

In [9]:
def SentenceTransformerSum(section, n_sents=50, method='kmeans'):
    sentences = preprocessed_list_of_sentences(section)
    embeddings = transformer.encode(sentences)
    if embeddings.size == 0:
        logging.warning("strategy is empty")
        return ''
    if len(sentences) < n_sents:
        logging.warning("Overview longer than strategy")
        return 'Overview was longer than strategy'
    
    if method == 'kmedoids':
        kmedoids = KMedoids(n_clusters=n_sents)
        kmedoids.fit(embeddings)
        summary = summaryFromClusters(kmedoids, embeddings, sentences, method='kmedoids')
    elif method == 'kmeans':
        clusterer = KMeans(n_clusters=n_sents)
        embeddings = normalize(embeddings, norm='l2')
        clusterer.fit(embeddings)
        summary = summaryFromClusters(clusterer, embeddings, sentences, method='kmeans')
    
    return summary

    
def Aggl_SentenceTransformerSum(section, n_sents=50):
    sentences = preprocessed_list_of_sentences(section)
    embeddings = transformer.encode(sentences)
    if embeddings.size == 0:
        return ''
    if len(sentences) < n_sents:
        return 'Overview was longer than strategy'
    
    embeddings = normalize(embeddings, norm='l2')
    clusterer = AgglomerativeClustering(n_clusters=n_sents, affinity='eauclidean', linkage='ward')
    clusterer.fit(embeddings)
    
    summ = summaryFromClusters(clusterer, embeddings, sentences)
    return summ

In [10]:
def universalEmbeddingsSum(section, n_sents=50, method='kmeans'):
    sentences = preprocessed_list_of_sentences(section)
    if len(sentences) == 0:
        return ''
    with tf.Session() as session:
        session.run([tf.global_variables_initializer(), tf.tables_initializer()])
        embeddings = session.run(embed(sentences))
    if embeddings.shape[0] == 0:
        logging.warning("Strategy is empty")
        return ''
    if len(sentences) < n_sents:
        logging.warning("Overview longer thans strategy.")
        return "Overview is longer than Strategy"
    
    if method == 'kmedoids':
        kmedoids = KMedoids(n_clusters=n_sents)
        kmedoids.fit(embeddings)
        summary = summaryFromClusters(kmedoids, embeddings, sentences, method='kmedoids')
    elif method == 'kmeans':
        clusterer = KMeans(n_clusters=n_sents)
        embeddings = normalize(embeddings, norm='l2')
        clusterer.fit(embeddings)
        summary = summaryFromClusters(clusterer, embeddings, sentences, method='kmeans')    
    return summary

    
def Aggl_universalEmbeddingsSum(section, n_sents=50):
    sentences = preprocessed_list_of_sentences(section)
    if len(sentences) == 0:
        return ''
    with tf.Session() as session:
        session.run([tf.global_variables_initializer(), tf.tables_initializer()])
        embeddings = session.run(embed(sentences))
    if embeddings.shape[0] == 0:
        return ''
    if len(sentences) < n_sents:
        return "Overview is longer than Strategy"
    
    embeddings = normalize(embeddings, norm='l2')
    clusterer = AgglomerativeClustering(n_clusters=n_sents, affinity='euclidean', linkage='ward')
    clusterer.fit(embeddings)
    
    summary = summaryFromClusters(clusterer, embeddings, sentences)
    return summary

In [5]:
import torch
device = torch.device("cuda")
print(device)

cuda


In [14]:
from transformers import RobertaForMaskedLM, RobertaTokenizer
#from utils import encoder, get_paranmt_usif
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
roberta = RobertaForMaskedLM.from_pretrained("roberta-base")
best_state = torch.load("./RobertaCheckpoints/best_state.pt")
roberta.load_state_dict(best_state['model_state'])
roberta.to(device)

def generate_sif(sentences, vectorizer, max_len, alpha=0.001):
    sparse_mtrx = vectorizer.fit_transform(sentences)
    print(sparse_mtrx.shape)
    token_weights = sparse_mtrx.sum(axis=0)
    print(token_weights.shape)
    weights_sum = sparse_mtrx.sum()
    features = vectorizer.get_feature_names()
    weights = torch.zeros((len(sentences), max_len))
    for j, sentence in enumerate(sentences):
        tokens = tokenizer.tokenize(sentence)
        for i in range(1, min(len(tokens) + 1, max_len)):
            if not tokens[i-1] in features:
                print(tokens[i-1])
                continue
            token_id = features.index(tokens[i-1])
            pw = token_weights[0, token_id] / weights_sum
            weight = alpha / (alpha + pw)
            weights[j, i] = weight
    
    return weights
            
def generate_tfidf(sentences, vectorizer, max_len):
    sparse_mtrx = vectorizer.fit_transform(sentences)
    features = vectorizer.get_feature_names()
    weights = torch.zeros((len(sentences), max_len))
    for j, sentence in enumerate(sentences):
        tokens = tokenizer.tokenize(sentence)
        for i in range(1, min(len(tokens) + 1, max_len)):
            if not tokens[i-1] in features:
                print(tokens[i-1])
                continue
            token_id = features.index(tokens[i-1])
            weight = sparse_mtrx[j, token_id] 
            weights[j, i] = weight
    
    return weights

def compute_hierarchy(vector):
    n_words = vector.size(0)
    avrgs = []
    for i in range(n_words-2):
        avrg = vector[i:i+3, :].mean(0)
        avrgs.append(avrg)
    avrgs = torch.stack(avrgs, dim=0)
    return avrgs.max(dim=0).values

def kde(embedding, n_samples, seed=42):
    """
    embedding is of shape (n_words, dim)
    """
    embedding = embedding.numpy()
    samples = []
    for i in range(embedding.shape[1]):
        dim = embedding[:, i:i+1]
        model = KernelDensity().fit(dim)
        samples.append(model.sample(n_samples, random_state=seed))
    samples = np.concatenate(samples, axis=1)
    samples = np.ravel(samples)
    return torch.from_numpy(samples)

def pmean(X, n=3):
    X1 = torch.mean(X**(-1), dim=0).squeeze()
    X2 = torch.mean(X**(2), dim=0).squeeze()
    X  = torch.mean(X, dim=0).squeeze()
    return torch.cat((X1, X, X2))

def Umap(embedding):
    umap_reducer = umap.UMAP(n_components=3, n_neighbors=25, min_dist=0.15, metric='cosine', random_state=42)
    embedding = torch.from_numpy(umap_reducer.fit_transform(embedding.T))
    embedding = np.ravel(embedding.T)
    return torch.from_numpy(embedding)
def VLAWE(embedding):
    kmeans = KMeans(n_clusters=3)
    embedding = embedding.numpy()
    kmeans.fit(embedding)
    codebooks = kmeans.cluster_centers_
    labels = kmeans.labels_
    vlawe = []
    for i, codebook in enumerate(codebooks):
        points = embedding[labels == i]
        vector = points - codebook.reshape(1, -1)
        vector = np.sum(vector, axis=0)
        vlawe.append(vector)
    return torch.from_numpy(np.concatenate(vlawe))

def DCT(embedding, n=3):
    n_words = embedding.shape[0]
    dims = embedding.shape[1]
    transformed = np.zeros((n, dims))
    for dim in range(dims):
        vector = embedding[:, dim].numpy()
        out = dct(vector, n = n, norm='ortho')
        transformed[:, dim] = out
    return torch.from_numpy(np.ravel(transformed))
def RobertaSum(text, model=roberta, n_sents=50, method='kmeans'):
    with torch.no_grad():
        sentences = preprocessed_list_of_sentences(text)
        inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
        max_len = inputs['input_ids'].size(1)
        inputs = torch.unbind(inputs['input_ids'])
        if len(sentences) < n_sents:
            logging.warning("Overview larger than strategy.")
            return 'Overview is larger than the Strategy.'
        
        embeddings = []
        weights = generate_sif(sentences, CountVectorizer(tokenizer=tokenizer.tokenize), max_len=max_len, alpha=0.001)
        for i, inpt in enumerate(inputs):
            scores, hiddens = model(inpt.unsqueeze(0).to(device), output_hidden_states=True)
            ##############Start of hierarchical##############################
#             embedding = compute_hierarchy(hiddens[0].squeeze().cpu())
#             embeddings.append(embedding)
#             #############Start Calculating SIF Embeddings#############################
            hidden = hiddens[0].squeeze().cpu()
            weight = weights[i, :].unsqueeze(0)
            if weight.sum().item() == 0:
                sentences.pop(i)
                continue
            embedding = torch.matmul(weight, hidden)/ weight.sum().item()
            embeddings.append(embedding)
            #to be continued after full doc embedding below:
#           ###################Start of TFIDF#################################################
#             hidden = hiddens[0].squeeze().cpu()
#             weight = weights[i, :].unsqueeze(0)
#             if weight.sum().item() == 0:
#                 sentences.pop(i)
#                 continue
#             embedding = torch.matmul(weight, hidden)/ weight.sum().item()
#             embeddings.append(embedding)
#           ###################End of Weighted Embeddings#####################################
            ###################Start Max Embeddings###########################################
#             embedding = hiddens[0].squeeze().cpu()
#             embedding = torch.mean(embedding, dim=0).squeeze()
#             embeddings.append(embedding)
            #######################End of Max Embeddings#######################################
            ##################Start of Power mean#############################################
#             embedding = hiddens[0].squeeze().cpu()
#             embedding = pmean(embedding)
#             embeddings.append(embedding)
            #################end of power mean################################################
            #######################Start of KDE################################################
#             embedding = hiddens[0].squeeze().cpu()
#             n_samples = 3
#             embedding = kde(embedding, n_samples)
#             embeddings.append(embedding)
#             #######################End of KDE###################################################
              #######################Start of Umap################################################
#             embedding = hiddens[0].squeeze().cpu()
#             embedding = Umap(embedding)
#             embeddings.append(embedding)
              #######################End of Umap###################################################
#             embedding = hiddens[0].squeeze().cpu()
#             embedding = VLAWE(embedding)
#             embeddings.append(embedding)
            ########################Start of DCT####################################################
#             embedding = hiddens[0].squeeze().cpu()
#             embedding = DCT(embedding)
#             embeddings.append(embedding)
            ########################END of DCT#####################################################
        embeddings = torch.stack(embeddings).squeeze().numpy()
        ##############################Continue of SIF##########################################
        svd = TruncatedSVD(n_components=1, n_iter=5, random_state=42)
        pc = svd.fit(embeddings).components_
        embeddings = embeddings - embeddings.dot(pc.transpose()) * pc
          ########################################################################################
    #################################Start of uSIF##############################################
        #embeddings = get_paranmt_usif(sentences, model, tokenizer, CountVectorizer(tokenizer=tokenizer.tokenize))
    ##################################End of uSIF###############################################
        if method == 'kmedoids':
            kmedoids = KMedoids(n_clusters=n_sents)
            kmedoids.fit(embeddings)
            summary = summaryFromClusters(kmedoids, embeddings, sentences, method='kmedoids')
        elif method == 'kmeans':
            embeddings = normalize(embeddings, norm='l2')
            clusterer = KMeans(n_clusters=n_sents)
            clusterer.fit(embeddings)
            summary = summaryFromClusters(clusterer, embeddings, sentences, method='kmeans')
        return summary

def Aggl_Roberta(text, model=roberta, n_sents=50):
    with torch.no_grad():
        sentences = preprocessed_list_of_sentences(text)
        inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
        inputs = torch.unbind(inputs['input_ids'])
        
        if len(sentences) < n_sents:
            return 'Overview is larger than the Strategy.'
        embeddings = []
        for inpt in inputs:
            scores, hiddens = model(inpt.unsqueeze(0).to(device), output_hidden_states=True)
            hidden = hiddens[0].squeeze()
            embedding = hidden.mean(dim=0)
            embeddings.append(embedding)
        
        embeddings = torch.stack(embeddings).cpu()
        embeddings = normalize(embeddings.numpy(), norm='l2')
        clusterer = AgglomerativeClustering(n_clusters=n_sents, affinity='euclidean', linkage='ward')
        clusterer.fit(embeddings)
        
        summary = summaryFromClusters(clusterer, embeddings, sentences)
        return summary

Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at roberta-base and are newly initialized: ['lm_head.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
def write_to_file(text, address):
    lines = textwrap.wrap(text, 100)
    output = '\n'.join(lines)
    with open(address, 'w') as f:
        f.write(output)

In [14]:
for item in glob.iglob("42ReportsAndSummaries/*_strategy.txt"):
    if item.endswith("SkipThouhgtsSum.txt") or item.endswith("SentenceTransformerSum.txt") or\
        item.endswith("UniversalEmbeddingsSum.txt") or item.endswith("RobertaSum.txt") or item.endswith("_overview.txt"):
        continue
    print(item)
    with open(os.path.dirname(item) + "/" + os.path.basename(item)[:-13] + "_overview.txt") as f:
        overview = f.read()
    
    n_sents = len(preprocessed_list_of_sentences(overview))
    if n_sents == 0 :
        continue
    
    with open(item, 'r', encoding='utf-8', errors='ignore') as f:
        section = f.read()
    if not section:
        continue
#     if not os.path.isfile(os.path.dirname(item) + "/summaries Aug 21-20/clustering/" + os.path.basename(item)[:-13] + "Kmeans_SkipThouhgtsSum.txt"):
#         skipthoughtsum = SkipThoughtsSum(section, n_sents=n_sents, method='kmeans')
#         write_to_file(skipthoughtsum, os.path.dirname(item) + "/summaries Aug 21-20/clustering/" + os.path.basename(item)[:-13] + "Kmeans_SkipThouhgtsSum.txt")
  
#     if not os.path.isfile(os.path.dirname(item) + "/summaries Aug 21-20/clustering/" + os.path.basename(item)[:-13] + "Kmeans_SentenceTransformerSum.txt"):
#         sentenceTransformerSum = SentenceTransformerSum(section, n_sents=n_sents, method='kmeans')
#         write_to_file(sentenceTransformerSum, os.path.dirname(item) + "/summaries Aug 21-20/clustering/" + os.path.basename(item)[:-13] + "Kmeans_SentenceTransformerSum.txt")

#     if not os.path.isfile(os.path.dirname(item) + "/summaries Aug 21-20/clustering/" + os.path.basename(item)[:-13] + "Kmeans_UniversalEmbeddingsSum.txt"):
#         uniEmbeddingsSum = universalEmbeddingsSum(section, n_sents=n_sents, method='kmeans')
#         write_to_file(uniEmbeddingsSum, os.path.dirname(item) + "/summaries Aug 21-20/clustering/" + os.path.basename(item)[:-13] + "Kmeans_UniversalEmbeddingsSum.txt")
  
    if not os.path.isfile(os.path.dirname(item) + "/summaries Aug 21-20/clustering/" + os.path.basename(item)[:-13] + "Kmeans_DCIRobertaSum.txt"):
        robertaSum = RobertaSum(section, n_sents=n_sents, method='kmeans')
        write_to_file(robertaSum, os.path.dirname(item) + "/summaries Aug 21-20/clustering/" + os.path.basename(item)[:-13] + "Kmeans_DCTRobertaSum.txt")
  

42ReportsAndSummaries/ADMIRAL GROUP PLC REPORT AND ACCOUNTS 2015_strategy.txt
42ReportsAndSummaries/ABERFORTH SMALLER COMPANIES TRUST PLC REPORT AND ACCOUNTS 2015_strategy.txt
42ReportsAndSummaries/ACACIA MINING PLC REPORT AND ACCOUNTS 2015_strategy.txt
42ReportsAndSummaries/VIRGIN MONEY HOLDINGS (UK) PLC REPORT AND ACCOUNTS 2015_strategy.txt
42ReportsAndSummaries/AMLIN PLC REPORT AND ACCOUNTS 2014_strategy.txt
42ReportsAndSummaries/AG BARR PLC REPORT AND ACCOUNTS 2015_strategy.txt
42ReportsAndSummaries/WHITBREAD PLC REPORT AND ACCOUNTS 2017_strategy.txt
42ReportsAndSummaries/WOODFORD PATIENT CAPITAL TRUST PLC REPORT AND ACCOUNTS 2015_strategy.txt
42ReportsAndSummaries/WORKSPACE GROUP PLC REPORT AND ACCOUNTS 2015_strategy.txt
42ReportsAndSummaries/ALLIED MINDS PLC REPORT AND ACCOUNTS 2015_strategy.txt
42ReportsAndSummaries/WH SMITH PLC REPORT AND ACCOUNTS 2015_strategy.txt
42ReportsAndSummaries/AA PLC REPORT AND ACCOUNTS 2015_strategy.txt
42ReportsAndSummaries/ZOOPLA PROPERTY GROUP PLC

### Here on the rouge scores are calculated
We are only calculating rouge1, rouge2, rougeL


We dump them all in a dictionary and then into a .xlsx file

In [8]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'])

In [9]:
def compute_rouge(summary, reference, scorer):
    score = scorer.score(summary, reference)
    #rougeprecision = round(score[rouge].precision, 2)
    rouge1 = round(score['rouge1'].recall, 4)
    rouge2 = round(score['rouge2'].recall, 4)
    rougeL = round(score['rougeL'].recall, 4)
    #rougefmeasure = round(score[rouge].fmeasure, 2)
    #rouge_string = str(rouge1) + "||" + str(rouge2) + "||" + str(rougeL)

    return rouge1, rouge2, rougeL

In [19]:
scores_table = { 
                ('SkipThouhgts', 'Rouge1'): [], ('SkipThouhgts', 'Rouge2'): [], ('SkipThouhgts', 'RougeL'): [],
                ('SentenceTransformer', 'Rouge1'): [], ('SentenceTransformer', 'Rouge2'): [], ('SentenceTransformer', 'RougeL'): [],
                ('UniversalEmbeddings', 'Rouge1'): [], ('UniversalEmbeddings', 'Rouge2'): [], ('UniversalEmbeddings', 'RougeL'): [],
                ('PmeanRoberta', 'Rouge1'): [], ('PmeanRoberta', 'Rouge2'): [], ('PmeanRoberta', 'RougeL'): []
               }
company = []
for item in glob.iglob("42ReportsAndSummaries/*_overview.txt"):
    company_name = os.path.basename(item)[:-13]
    with open(item, encoding='utf-8') as f:
        reference = f.read()
    summaries_l = [company_name + "Kmeans_SkipThouhgtsSum.txt",
                   company_name + "Kmeans_SentenceTransformerSum.txt",
                   company_name + "Kmeans_UniversalEmbeddingsSum.txt",
                   company_name + "Kmeans_PmeanRobertaSum.txt"]

    #scores_table['company'].append(company_name)
    company.append(company_name)
    for sumname in summaries_l:
        try:
            with open("42ReportsAndSummaries/summaries Aug 21-20/clustering/" + sumname, encoding='utf-8') as f:
                summary = f.read()
        except FileNotFoundError:
            summary = ''

        rouge1, rouge2, rougeL = compute_rouge(summary, reference, scorer)
        scores_table[(sumname.split('_')[1][:-7], 'Rouge1')].append(rouge1)
        scores_table[(sumname.split('_')[1][:-7], 'Rouge2')].append(rouge2)
        scores_table[(sumname.split('_')[1][:-7], 'RougeL')].append(rougeL)

In [20]:
scores_table = pd.DataFrame(scores_table)
scores_table = scores_table.append(pd.DataFrame(scores_table.apply(np.mean)).T).set_index(pd.Index(company + ['Average']))
scores_table.to_csv("42ReportsAndSummaries/summaries Aug 21-20/Kmeans_PmeanRoberta.csv")
scores_table

SkipThouhgts            \
                                                         Rouge1    Rouge2   
WIZZ AIR HOLDINGS PLC REPORT AND ACCOUNTS 2016         0.694400  0.234600   
VINACAPITAL VIETNAM OPPORTUNITY FUND LTD REPORT...     0.628100  0.149400   
3I INFRASTRUCTURE PLC REPORT AND ACCOUNTS 2015         0.511800  0.088800   
VIETNAM ENTERPRISE INVESTMENTS LTD REPORT AND A...     0.683700  0.268000   
AGGREKO PLC REPORT AND ACCOUNTS 2015                   0.596300  0.129600   
WITAN INVESTMENT TRUST PLC REPORT AND ACCOUNTS ...     0.715200  0.164600   
WOLSELEY PLC REPORT AND ACCOUNTS 2015                  0.528400  0.097100   
ABERFORTH SMALLER COMPANIES TRUST PLC REPORT AN...     0.833300  0.200000   
AMEC FOSTER WHEELER PLC REPORT AND ACCOUNTS 2015       0.696200  0.216600   
WHITBREAD PLC REPORT AND ACCOUNTS 2017                 0.525300  0.122400   
AG BARR PLC REPORT AND ACCOUNTS 2015                   0.503100  0.139200   
ABERDEEN ASSET MANAGEMENT PLC REPORT AND ACCOUN...     0.828000  0.239100   
ALLIANCE TRUST PLC REPORT AND ACCOUNTS 2015            0.607100  0.096400   
WORLDWIDE HEALTHCARE TRUST PLC REPORT AND ACCOU...     0.736600  0.304900   
WM MORRISON SUPERMARKETS PLC REPORT AND ACCOUNT...     0.777800  0.375000   
ADMIRAL GROUP PLC REPORT AND ACCOUNTS 2015             0.713500  0.265500   
WILLIAM HILL PLC REPORT AND ACCOUNTS 2015              0.641400  0.131900   
WH SMITH PLC REPORT AND ACCOUNTS 2015                  0.538500  0.124200   
VEDANTA RESOURCES PLC REPORT AND ACCOUNTS 2015         0.630900  0.135100   
AA PLC REPORT AND ACCOUNTS 2015                        0.653800  0.116900   
WOODFORD PATIENT CAPITAL TRUST PLC REPORT AND A...     0.734800  0.219400   
VIRGIN MONEY HOLDINGS (UK) PLC REPORT AND ACCOU...     0.946900  0.653300   
WORLDPAY GROUP PLC REPORT AND ACCOUNTS 2015            0.543500  0.131900   
AMLIN PLC REPORT AND ACCOUNTS 2014                     0.690600  0.137700   
ANGLO AMERICAN PLC REPORT AND ACCOUNTS 2015            0.669700  0.148100   
ACACIA MINING PLC REPORT AND ACCOUNTS 2015             0.649700  0.171200   
3I GROUP PLC REPORT AND ACCOUNTS 2015                  0.701300  0.105300   
ZOOPLA PROPERTY GROUP PLC REPORT AND ACCOUNTS 2015     0.761900  0.411000   
WORKSPACE GROUP PLC REPORT AND ACCOUNTS 2015           0.777800  0.352900   
WETHERSPOON (JD) PLC REPORT AND ACCOUNTS 2016          0.680600  0.216800   
WEIR GROUP PLC REPORT AND ACCOUNTS 2015                0.596600  0.127100   
ALLIED MINDS PLC REPORT AND ACCOUNTS 2015              0.377100  0.059800   
VODAFONE GROUP PLC REPORT AND ACCOUNTS 2015            0.700700  0.226000   
ALDERMORE GROUP PLC REPORT AND ACCOUNTS 2015           0.752400  0.259600   
888 HOLDINGS PLC REPORT AND ACCOUNTS 2017              0.540000  0.128500   
Average                                                0.661914  0.198511   

                                                              \
                                                      RougeL   
WIZZ AIR HOLDINGS PLC REPORT AND ACCOUNTS 2016      0.438900   
VINACAPITAL VIETNAM OPPORTUNITY FUND LTD REPORT...  0.380200   
3I INFRASTRUCTURE PLC REPORT AND ACCOUNTS 2015      0.258800   
VIETNAM ENTERPRISE INVESTMENTS LTD REPORT AND A...  0.530600   
AGGREKO PLC REPORT AND ACCOUNTS 2015                0.403700   
WITAN INVESTMENT TRUST PLC REPORT AND ACCOUNTS ...  0.381800   
WOLSELEY PLC REPORT AND ACCOUNTS 2015               0.261400   
ABERFORTH SMALLER COMPANIES TRUST PLC REPORT AN...  0.666700   
AMEC FOSTER WHEELER PLC REPORT AND ACCOUNTS 2015    0.430400   
WHITBREAD PLC REPORT AND ACCOUNTS 2017              0.313100   
AG BARR PLC REPORT AND ACCOUNTS 2015                0.245300   
ABERDEEN ASSET MANAGEMENT PLC REPORT AND ACCOUN...  0.505400   
ALLIANCE TRUST PLC REPORT AND ACCOUNTS 2015         0.392900   
WORLDWIDE HEALTHCARE TRUST PLC REPORT AND ACCOU...  0.433000   
WM MORRISON SUPERMARKETS PLC REPORT AND ACCOUNT...  0.666700   
ADMIRAL GROUP PLC REPORT AND ACCOUNT

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
a = np.random.randn(9, 1)
corpus = [
     'This is the first document.',
     'This document is the second document.',
     'And this is the third one.',
     'Is this the first document?',
]
vectorizer = TfidfVectorizer(tokenizer = tokenizer.tokenize)
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())
print(X.shape)
print(type(X))
print(X)
print(X.toarray())
print(X[1, 1])

In [ ]:
X.sum(axis=0)[0, 2]

In [ ]:
vectorizer.get_feature_names()

In [ ]:
x = generate_tfidf(corpus, vectorizer, max_len=9)
x

In [ ]:
from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
inputs = tokenizer(corpus, padding=True, truncation=True, return_tensors='pt')
inputs['input_ids']

In [ ]:
print(tokenizer.tokenize("This is annoyingly the first document. where we"))
print(word_tokenize("This's annoyingly the first document. where we"))

In [ ]:
sent_tokenize("Do you love transformers?I bet you do. How about this one.")

# Comparing Umaps 

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import umap
%matplotlib inline

In [19]:
reducer = umap.UMAP(random_state=42, n_neighbors=25)

section = extract_strategy("42ReportsAndSummaries/WM MORRISON SUPERMARKETS PLC REPORT AND ACCOUNTS 2015.txt")
overview = open("42ReportsAndSummaries/summaries Aug 21-20/WM MORRISON SUPERMARKETS PLC REPORT AND ACCOUNTS 2015_Overview.txt").read()
n_sents = len(preprocessed_list_of_sentences(overview))

vectors, sentences, labels, summary = SkipThoughtsSum(section, n_sents=n_sents)
reduced_vectors = reducer.fit_transform(vectors)

/home/mao919/anaconda3/envs/py37-start/lib/python3.7/site-packages/torch/nn/modules/rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.25 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
/home/mao919/anaconda3/envs/py37-start/lib/python3.7/site-packages/ipykernel_launcher.py:46: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (47). Possibly due to duplicate points in X.


In [26]:
strategy = extract_strategy("42ReportsAndSummaries/WM MORRISON SUPERMARKETS PLC REPORT AND ACCOUNTS 2015.txt")
#print(sent_tokenize(summary))
print(strategy)

Wm Morrison Supermarkets PLC Annual report and financial statements 2014/15
Governance Financial statements Strategic report The Morrisons difference
At Morrisons, we focus on fresh food. Our Market 
Street departments, and particularly our butchers, 
bakers, ﬁshmongers and greengrocers are skilled  
in a way no other supermarket in the UK can match, 
and are passionate about what they do. They prepare 
food the way our customers like it, and often to 
individual customer speciﬁcations. That focus on 
fresh food is present across the Group as well as in 
our new online and convenience businesses.
Our focus on fresh is supported by our vertically 
integrated ‘farm to fork’ business model. Unlike  
any other major UK supermarket, we manufacture 
more than half of the fresh food we sell ourselves, 
operating our own abattoirs and food production 
factories as well as our own farm for research and 
development purposes. We also operate our own 
• We are the second largest fresh food manufa

In [30]:
preprocessed_list_of_sentences(strategy)

['wm morrison supermarkets plc annual report and financial statements 2014/15 governance financial statements strategic report the morrisons difference at morrisons, we focus on fresh food.',
 'our market street departments, and particularly our butchers, bakers, fishmongers and greengrocers are skilled in a way no other supermarket in the uk can match, and are passionate about what they do.',
 'they prepare food the way our customers like it, and often to individual customer specifications.',
 'that focus on fresh food is present across the group as well as in our new online and convenience businesses.',
 'our focus on fresh is supported by our vertically integrated ‘farm to fork’ business model.',
 'unlike any other major uk supermarket, we manufacture more than half of the fresh food we sell ourselves, operating our own abattoirs and food production factories as well as our own farm for research and development purposes.',
 'we also operate our own • we are the second largest fresh 

In [ ]:
ax = plt.subplot(111)
ax.scatter(reduced_vectors[:, 0], reduced_vectors[:, 1], c=labels)

In [ ]:
reduced_senttransform, sentences = Kmeans_SentenceTransformerSum(section)
reduced_senttransform = reducer.fit_transform(reduced_senttransform)
plt.scatter(reduced_senttransform[:, 0], reduced_senttransform[:, 1])

In [ ]:
reduced_universembed, sentences = Kmeans_universalEmbeddingsSum(section)
reduced_universembed = reducer.fit_transform(reduced_universembed)
plt.scatter(reduced_universembed[:, 0], reduced_universembed[:, 1])

In [1]:
tokens = tokenizer.encode("lets test this")
tokens2 = tokenizer.encode("LEST test ThIs")
print(tokens)
print(tokens2)

NameError: name 'tokenizer' is not defined

In [24]:
tokenizer.convert_tokens_to_ids(tokens)

[10219, 1296, 42]

In [31]:
input = tokenizer("lets test this", padding=True, truncation=True, return_tensors='pt')
input

{'input_ids': tensor([[    0, 10219,  1296,    42,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}

In [34]:
scores, hiddens = roberta(input['input_ids'].to(device), output_hidden_states=True)
hiddens[0]

tensor([[[ 0.1634, -0.0544, -0.0040,  ..., -0.0806,  0.0655, -0.0028],
         [-0.4753, -0.0798,  0.7213,  ...,  0.2104, -0.0160,  0.2809],
         [-0.0142,  0.0169,  0.1685,  ...,  0.3775, -0.6792, -0.0541],
         [-0.0761,  0.0974,  0.0854,  ...,  0.2570, -0.3257,  0.2674],
         [ 0.2149, -0.2369, -0.1234,  ...,  0.4181,  0.0390,  0.2190]]],
       device='cuda:0', grad_fn=<NativeLayerNormBackward>)

In [39]:
hiddens[0].squeeze()

tensor([[ 0.1634, -0.0544, -0.0040,  ..., -0.0806,  0.0655, -0.0028],
        [-0.4753, -0.0798,  0.7213,  ...,  0.2104, -0.0160,  0.2809],
        [-0.0142,  0.0169,  0.1685,  ...,  0.3775, -0.6792, -0.0541],
        [-0.0761,  0.0974,  0.0854,  ...,  0.2570, -0.3257,  0.2674],
        [ 0.2149, -0.2369, -0.1234,  ...,  0.4181,  0.0390,  0.2190]],
       device='cuda:0', grad_fn=<SqueezeBackward0>)

In [40]:
hiddens[0].squeeze().shape

torch.Size([5, 768])

In [2]:
CV = CountVectorizer()
sparse_mtrx = CV.fit_transform(['This is the first sentence', 'this is the second sentence.'])
CV.vocabulary_

{'this': 5, 'is': 1, 'the': 4, 'first': 0, 'sentence': 3, 'second': 2}

In [3]:
for w in CV.vocabulary_:
    print(w)

this
is
the
first
sentence
second


In [7]:
print(sparse_mtrx.sum(axis=0))

[[1 2 1 2 2 2]]


In [35]:
np.squeeze(sparse_mtrx.sum(axis=0))

(1, 6)

In [4]:
sparse_mtrx.sum()

10